# 🔍 Project 8: Semantic Search & Recommendation System
### Lovely Professional University — Python & Full Stack Development

**Pipeline:** BBC News Dataset → Preprocessing → Word2Vec Embeddings → Semantic Search + Recommendations → PCA/t-SNE Visualization

---

In [ ]:
# Step 0: Install required libraries
!pip install gensim scikit-learn matplotlib seaborn pandas numpy kaggle --quiet
print("✅ All libraries installed.")

## 📦 Module 1: Data Collection & Preprocessing
We use the **BBC News Classification Dataset** from Kaggle — 2,225 real English news articles across 5 categories: *tech, business, sport, entertainment, politics*.

In [ ]:
# Step 1: Load BBC News dataset from the Kaggle folder structure
#
# HOW TO GET THE DATASET:
#   1. Go to: https://www.kaggle.com/datasets/shivamkushwaha/bbc-full-text-document-classification
#   2. Click Download → extract the zip
#   3. You will get a folder like: bbc-fulltext (document classification)/
#      Inside it: business/, tech/, sport/, entertainment/, politics/
#      Each subfolder has 001.txt, 002.txt, ... etc.
#   4. Place that top-level folder next to this notebook and set BBC_ROOT below.
#
# FOLDER STRUCTURE EXPECTED:
#   BBC_ROOT/
#     ├── business/  001.txt  002.txt ...
#     ├── tech/      001.txt  002.txt ...
#     ├── sport/     001.txt  002.txt ...
#     ├── entertainment/ ...
#     └── politics/  ...

import pandas as pd
import os
import glob

# ── SET THIS to your extracted folder path ────────────────────────────────────
BBC_ROOT = "bbc-fulltext (document classification)"   # <-- change if needed
# e.g. BBC_ROOT = "archive/bbc-fulltext (document c..."
# ─────────────────────────────────────────────────────────────────────────────

def load_bbc_folder(root_path):
    """Walk category subfolders and read every .txt file into a DataFrame."""
    records = []
    if not os.path.exists(root_path):
        return None
    for category in sorted(os.listdir(root_path)):
        cat_path = os.path.join(root_path, category)
        if not os.path.isdir(cat_path):
            continue
        for txt_file in sorted(glob.glob(os.path.join(cat_path, "*.txt"))):
            try:
                with open(txt_file, "r", encoding="utf-8", errors="replace") as f:
                    text = f.read().strip().replace("\n", " ")
                if text:  # skip empty files
                    records.append({"category": category, "text": text})
            except Exception as e:
                print(f"  ⚠️  Skipping {txt_file}: {e}")
    return pd.DataFrame(records) if records else None


df = load_bbc_folder(BBC_ROOT)

if df is not None and len(df) > 0:
    print(f"✅ Loaded BBC full-text dataset from folder: {df.shape[0]} articles")
    print(df["category"].value_counts())
else:
    print("⚠️  Folder not found or empty. Using built-in 100-article sample.")
    print(f"   Set BBC_ROOT to the path of your extracted Kaggle download.")
    print(f"   Current BBC_ROOT = '{BBC_ROOT}'")

    # ── Compact fallback sample ───────────────────────────────────────────────
    sample_data = [
        ("tech", "Broadband speeds slow as demand for internet access rises across UK homes and offices rapidly."),
        ("tech", "Mobile phone sales hit record high as smartphone manufacturers compete for global market share."),
        ("tech", "Google launches new search algorithm update affecting millions of websites worldwide."),
        ("tech", "Apple unveils new MacBook laptop with faster processor and improved battery life for users."),
        ("tech", "Microsoft releases major Windows security patch to fix critical vulnerabilities found by researchers."),
        ("tech", "Amazon Web Services expands cloud computing data centres across Europe and Asia regions."),
        ("tech", "Artificial intelligence startup raises funding to develop natural language processing systems."),
        ("tech", "Cybersecurity firms warn of increasing ransomware attacks targeting hospitals and governments."),
        ("tech", "Electric vehicle software updates now delivered wirelessly over the air to Tesla cars."),
        ("tech", "Quantum computing breakthrough achieved by IBM scientists using new superconducting qubit design."),
        ("tech", "Netflix invests in faster content delivery networks to improve streaming quality globally."),
        ("tech", "Social media platforms face new regulations requiring faster removal of harmful content."),
        ("tech", "Robot warehouse workers deployed by logistics companies to meet growing e-commerce demand."),
        ("tech", "Chip shortage continues to disrupt global electronics supply chains affecting car manufacturers."),
        ("tech", "5G network rollout accelerates as telecoms operators compete for customers in urban areas."),
        ("tech", "Open source software community grows as developers contribute to collaborative coding projects."),
        ("tech", "Virtual reality headsets become more affordable opening gaming and education applications widely."),
        ("tech", "Data privacy laws require tech firms to store user information within national borders."),
        ("tech", "Autonomous drones tested for delivery services in rural areas with limited road infrastructure."),
        ("tech", "Wearable fitness trackers now monitor blood oxygen levels and detect irregular heartbeats."),
        ("business", "Oil prices rise sharply as OPEC nations agree to reduce production output globally."),
        ("business", "Retail sales fall for third consecutive month amid rising household energy bills."),
        ("business", "Bank of England raises interest rates again to combat persistent inflation in economy."),
        ("business", "Stock markets tumble as investors worry about slowing economic growth in China."),
        ("business", "Major supermarket chain reports record profits driven by higher food prices consumers pay."),
        ("business", "Unemployment rate drops to lowest level in decades as job vacancies remain high."),
        ("business", "Budget airline announces new routes and increased capacity heading into summer travel season."),
        ("business", "Housing market slows as rising mortgage rates make property less affordable for buyers."),
        ("business", "Global shipping costs decline as supply chain disruptions from pandemic ease gradually."),
        ("business", "Luxury goods sales boom as wealthy consumers increase spending on travel and fashion."),
        ("business", "Mergers and acquisitions activity surges as companies seek growth through strategic takeovers."),
        ("business", "Government announces new trade deal expected to boost exports and create manufacturing jobs."),
        ("business", "Pension funds increase investment in renewable energy infrastructure and green bonds."),
        ("business", "Start-up funding falls sharply as venture capital firms become more cautious about valuations."),
        ("business", "Consumer confidence index rises as inflation begins to ease and wages grow faster."),
        ("business", "Company CEO resigns amid shareholder pressure over falling profits and strategic direction."),
        ("business", "Foreign direct investment flows into emerging markets despite global economic uncertainty."),
        ("business", "Automotive industry invests billions in electric vehicle production lines across Europe."),
        ("business", "E-commerce sales continue to grow as consumers prefer shopping online over physical stores."),
        ("business", "Cryptocurrency market rebounds after months of decline attracting new retail investors."),
        ("sport", "England cricket team wins test series against Australia in thrilling final match."),
        ("sport", "Premier League title race intensifies with three clubs separated by just two points."),
        ("sport", "Olympic gold medallist announces retirement from professional athletics after successful career."),
        ("sport", "Tennis grand slam champion injured during training casting doubt over upcoming tournament."),
        ("sport", "Football transfer window closes with record spending by top clubs across Europe."),
        ("sport", "Rugby world cup host nation prepares stadiums and infrastructure for major tournament."),
        ("sport", "Formula One driver secures pole position with fastest qualifying lap of the season."),
        ("sport", "Cycling team disqualified after doping violations found during post-race urine tests."),
        ("sport", "Basketball star signs record breaking contract extension with NBA franchise for five years."),
        ("sport", "Swimming world record broken at championships by teenager making international debut."),
        ("sport", "Horse racing festival attracts record crowds and highest prize money in event history."),
        ("sport", "Golf major champion defends title with dominant final round performance at Augusta."),
        ("sport", "National football team qualifies for World Cup after dramatic playoff victory away."),
        ("sport", "Boxing champion retains title after unanimous points decision in Las Vegas bout."),
        ("sport", "Stadium expansion project approved bringing capacity to over sixty thousand fans."),
        ("sport", "Women's football league reports record attendances and television viewing figures this season."),
        ("sport", "Marathon world record broken in Berlin with runner finishing under two hour mark."),
        ("sport", "Cricket board announces new twenty-twenty competition to attract younger global audiences."),
        ("sport", "Winter Olympics preparations completed as host city finalises athlete village facilities."),
        ("sport", "Snooker world champion beats rival in longest final frame ever recorded at Sheffield."),
        ("entertainment", "Hollywood blockbuster breaks box office records earning one billion dollars in opening week."),
        ("entertainment", "Streaming platform announces new original series featuring award-winning director and cast."),
        ("entertainment", "Music artist releases surprise album gaining millions of streams within first twenty-four hours."),
        ("entertainment", "Film awards ceremony dominated by independent productions rather than major studio releases."),
        ("entertainment", "Video game sells five million copies in first week becoming fastest selling title ever."),
        ("entertainment", "Famous actor receives lifetime achievement award at international film festival ceremony."),
        ("entertainment", "Television drama series finale watched by record audience of twenty million viewers."),
        ("entertainment", "Pop star announces world tour dates following multi-year absence from live performances."),
        ("entertainment", "Animated film wins best picture award surprising industry experts and film critics."),
        ("entertainment", "Theatre production transfers to Broadway after successful run in London West End."),
        ("entertainment", "Celebrity couple announces separation after ten years of marriage in joint statement."),
        ("entertainment", "Book becomes bestseller weeks before film adaptation premieres in cinemas worldwide."),
        ("entertainment", "Comedy special filmed live becomes most watched stand-up show on streaming platform."),
        ("entertainment", "Music festival lineup revealed featuring headliners from rock pop and electronic genres."),
        ("entertainment", "Reality television show returns for new series with major format changes announced."),
        ("entertainment", "Podcast breaks download record becoming most listened programme in platform history."),
        ("entertainment", "Documentary about climate change wins award and sparks international policy debate."),
        ("entertainment", "Rapper collaborates with classical orchestra producing genre-defying album to critical acclaim."),
        ("entertainment", "Subscription costs rise at major streaming services causing viewer cancellations to increase."),
        ("entertainment", "Art exhibition draws record visitors after rare paintings go on display publicly."),
        ("politics", "Prime minister announces snap election amid falling poll ratings and party internal divisions."),
        ("politics", "Parliament debates new immigration bill as opposition parties demand significant amendments."),
        ("politics", "Foreign minister meets counterpart to discuss trade relations and diplomatic cooperation."),
        ("politics", "Government faces vote of no confidence after corruption allegations made against ministers."),
        ("politics", "United Nations security council votes on resolution addressing humanitarian crisis overseas."),
        ("politics", "Political party leadership contest begins after former leader resigns following election defeat."),
        ("politics", "New climate legislation passed requiring carbon emissions reductions by thirty percent."),
        ("politics", "Referendum results show narrow majority in favour of constitutional reform proposals."),
        ("politics", "Diplomatic tensions rise between neighbouring countries over disputed border territory."),
        ("politics", "Budget statement outlines tax increases and spending cuts to reduce government deficit."),
        ("politics", "Opposition leader criticises government handling of national health service funding crisis."),
        ("politics", "International sanctions imposed on country following violations of human rights agreements."),
        ("politics", "Local council elections produce surprise results shifting political balance in key regions."),
        ("politics", "President signs executive order addressing national security threats from foreign actors."),
        ("politics", "Peace talks begin between rival factions with international mediators facilitating negotiations."),
        ("politics", "Whistleblower reveals confidential government documents causing major political scandal."),
        ("politics", "Election commission investigates alleged campaign finance violations by major party."),
        ("politics", "New transport infrastructure bill promises investment in roads railways and digital networks."),
        ("politics", "Protests erupt in capital city as citizens demonstrate against proposed austerity measures."),
        ("politics", "Coalition government collapses as junior partner withdraws support over policy disagreement."),
    ]
    df = pd.DataFrame(sample_data, columns=["category", "text"])
    print(f"✅ Fallback sample loaded: {len(df)} articles, {df['category'].nunique()} categories.")

# Reset index so iloc lookups are consistent
df = df.reset_index(drop=True)
print(f"\nFinal dataset shape : {df.shape}")
df.head()

In [ ]:
# Step 2: Text preprocessing
import re
import string

# Simple stopword list (avoiding NLTK download requirement)
STOPWORDS = set([
    "a","an","the","and","or","but","in","on","at","to","for","of","with",
    "by","from","as","is","was","are","were","be","been","being","have",
    "has","had","do","does","did","will","would","could","should","may",
    "might","shall","can","it","its","this","that","these","those","i",
    "he","she","we","they","you","me","him","her","us","them","my","his",
    "their","our","your","not","no","nor","so","yet","both","either",
    "each","few","more","most","other","some","such","than","too","very",
    "just","also","about","after","before","between","into","through",
    "during","over","up","down","out","off","then","once","here","there",
    "when","where","how","all","any","new","said","say","says"
])

def preprocess(text):
    """Lowercase → remove punctuation → tokenize → remove stopwords."""
    text = text.lower()
    text = re.sub(r"[^a-z\s]", "", text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in STOPWORDS and len(t) > 2]
    return tokens

df["tokens"] = df["text"].apply(preprocess)

print("Sample preprocessing:")
print(f"  Original : {df['text'][0]}")
print(f"  Tokens   : {df['tokens'][0]}")
print(f"  Count    : {len(df['tokens'][0])} tokens")

## 🧠 Module 2: Word2Vec Embedding Generation
We train a **Word2Vec Skip-Gram** model from scratch on the BBC corpus, then produce sentence-level vectors by **mean-pooling** word vectors.

In [ ]:
# Step 3: Train Word2Vec model on the BBC corpus
from gensim.models import Word2Vec

sentences = df["tokens"].tolist()  # list of token lists

w2v_model = Word2Vec(
    sentences=sentences,
    vector_size=100,    # embedding dimensions
    window=5,           # context window size
    min_count=1,        # include all words
    sg=1,               # 1 = Skip-Gram, 0 = CBOW
    epochs=100,         # training epochs
    seed=42
)

print(f"✅ Word2Vec trained successfully!")
print(f"   Vocabulary size : {len(w2v_model.wv.key_to_index)} words")
print(f"   Vector size     : {w2v_model.vector_size} dimensions")
print(f"   Training mode   : Skip-Gram")

In [ ]:
# Step 4: Explore learned word vectors
import numpy as np

# Most similar words for selected seed words
seed_words = ["technology", "football", "government", "market", "film"]

for word in seed_words:
    if word in w2v_model.wv:
        similar = w2v_model.wv.most_similar(word, topn=4)
        pairs = ", ".join([f"{w} ({s:.2f})" for w, s in similar])
        print(f"  '{word}' → {pairs}")
    else:
        print(f"  '{word}' not in vocabulary")

In [ ]:
# Step 5: Generate sentence vectors via mean-pooling of word vectors
def sentence_vector(tokens, model):
    """Average the Word2Vec vectors of all known tokens in a sentence."""
    vecs = [model.wv[t] for t in tokens if t in model.wv]
    if not vecs:
        return np.zeros(model.vector_size)
    return np.mean(vecs, axis=0)

# Build sentence embedding matrix  shape: (N x 100)
sentence_vectors = np.array([sentence_vector(tok, w2v_model) for tok in df["tokens"]])
print(f"Sentence embedding matrix shape : {sentence_vectors.shape}")
print(f"Sample vector (first 8 dims)    : {sentence_vectors[0][:8].round(4)}")

## 🔍 Module 3A: Semantic Search
Given a **query sentence**, encode it with the same Word2Vec pipeline and rank all articles by **cosine similarity**.

In [ ]:
# Step 6: Cosine similarity helper
from sklearn.metrics.pairwise import cosine_similarity

def semantic_search(query, df, sentence_vectors, w2v_model, top_n=5):
    """
    Encode query → compute cosine similarity vs all articles
    → return top-N results with similarity scores.
    """
    query_tokens = preprocess(query)
    query_vec    = sentence_vector(query_tokens, w2v_model).reshape(1, -1)

    sims = cosine_similarity(query_vec, sentence_vectors)[0]

    top_indices = sims.argsort()[::-1][:top_n]
    results = []
    for rank, idx in enumerate(top_indices, 1):
        results.append({
            "Rank"      : rank,
            "Score"     : round(float(sims[idx]), 4),
            "Category"  : df["category"].iloc[idx],
            "Article"   : df["text"].iloc[idx]
        })
    return pd.DataFrame(results)

# ── Run three example queries ─────────────────────────────────────────────────
queries = [
    "smartphone internet mobile technology",
    "football match premier league",
    "economy inflation interest rates"
]

for q in queries:
    print(f"\n{'─'*65}")
    print(f"🔎 Query: '{q}'")
    print(f"{'─'*65}")
    results = semantic_search(q, df, sentence_vectors, w2v_model, top_n=3)
    for _, row in results.iterrows():
        print(f"  #{row['Rank']} [{row['Category'].upper()}] (score={row['Score']})")
        print(f"     {row['Article']}")

## 🎯 Module 3B: Recommendation System
Given an **article index**, find the most similar articles across the corpus — article-to-article recommendation.

In [ ]:
# Step 7: Article-to-article recommendation
def recommend(article_idx, df, sentence_vectors, top_n=5):
    """
    Given an article index, return top-N most similar articles
    (excluding the article itself).
    """
    query_vec = sentence_vectors[article_idx].reshape(1, -1)
    sims      = cosine_similarity(query_vec, sentence_vectors)[0]

    # Exclude the article itself
    sims[article_idx] = -1
    top_indices = sims.argsort()[::-1][:top_n]

    results = []
    for rank, idx in enumerate(top_indices, 1):
        results.append({
            "Rank"     : rank,
            "Score"    : round(float(sims[idx]), 4),
            "Category" : df["category"].iloc[idx],
            "Article"  : df["text"].iloc[idx]
        })
    return pd.DataFrame(results)


# ── Demonstrate on 3 seed articles ───────────────────────────────────────────
seed_articles = [0, 20, 40]   # tech, business, sport

for seed in seed_articles:
    print(f"\n{'═'*65}")
    print(f"📰 SEED ARTICLE [{df['category'].iloc[seed].upper()}]")
    print(f"   {df['text'].iloc[seed]}")
    print(f"{'─'*65}")
    print("   📌 TOP RECOMMENDATIONS:")
    recs = recommend(seed, df, sentence_vectors, top_n=4)
    for _, row in recs.iterrows():
        print(f"   #{row['Rank']} [{row['Category'].upper()}] (score={row['Score']})")
        print(f"      {row['Article']}")

## 📊 Module 4: Visualization
We project 100-dimensional sentence vectors down to 2D using **PCA** and **t-SNE** to visually inspect clustering by news category.

In [ ]:
# Step 8: PCA visualization (2D)
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.decomposition import PCA

CATEGORY_COLORS = {
    "tech"          : "#4E79A7",
    "business"      : "#F28E2B",
    "sport"         : "#59A14F",
    "entertainment" : "#E15759",
    "politics"      : "#B07AA1"
}

pca = PCA(n_components=2, random_state=42)
pca_coords = pca.fit_transform(sentence_vectors)

fig, ax = plt.subplots(figsize=(10, 7))
for cat, color in CATEGORY_COLORS.items():
    mask = df["category"] == cat
    ax.scatter(
        pca_coords[mask, 0], pca_coords[mask, 1],
        c=color, label=cat.capitalize(), alpha=0.85,
        edgecolors="white", linewidths=0.6, s=80
    )

ax.set_title("PCA — BBC News Articles by Category (Word2Vec Embeddings)",
             fontsize=14, fontweight="bold", pad=12)
ax.set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)")
ax.set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)")
ax.legend(title="Category", framealpha=0.9)
ax.grid(True, linestyle="--", alpha=0.3)
plt.tight_layout()
plt.savefig("pca_visualization.png", dpi=150, bbox_inches="tight")
plt.show()
print(f"PCA explains {sum(pca.explained_variance_ratio_)*100:.1f}% of total variance.")

In [ ]:
# Step 9: t-SNE visualization (2D) — better non-linear cluster separation
from sklearn.manifold import TSNE

# Perplexity must be < number of samples
perplexity = min(30, len(df) - 1)

tsne = TSNE(n_components=2, perplexity=perplexity, random_state=42,
            n_iter=1000, learning_rate="auto", init="pca")
tsne_coords = tsne.fit_transform(sentence_vectors)

fig, ax = plt.subplots(figsize=(10, 7))
for cat, color in CATEGORY_COLORS.items():
    mask = df["category"] == cat
    ax.scatter(
        tsne_coords[mask, 0], tsne_coords[mask, 1],
        c=color, label=cat.capitalize(), alpha=0.85,
        edgecolors="white", linewidths=0.6, s=80
    )

ax.set_title("t-SNE — BBC News Articles by Category (Word2Vec Embeddings)",
             fontsize=14, fontweight="bold", pad=12)
ax.set_xlabel("t-SNE Dimension 1")
ax.set_ylabel("t-SNE Dimension 2")
ax.legend(title="Category", framealpha=0.9)
ax.grid(True, linestyle="--", alpha=0.3)
plt.tight_layout()
plt.savefig("tsne_visualization.png", dpi=150, bbox_inches="tight")
plt.show()
print("t-SNE visualization saved.")

In [ ]:
# Step 10: Cosine similarity heatmap — 20 sample articles
import seaborn as sns

# Pick 4 from each category for a balanced heatmap
sample_idx = []
for cat in ["tech", "business", "sport", "entertainment", "politics"]:
    idxs = df[df["category"] == cat].index.tolist()[:4]
    sample_idx.extend(idxs)

sample_vecs   = sentence_vectors[sample_idx]
sim_matrix    = cosine_similarity(sample_vecs)
sample_labels = [f"{df['category'].iloc[i][:3].upper()}-{i}" for i in sample_idx]

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(
    sim_matrix,
    xticklabels=sample_labels, yticklabels=sample_labels,
    cmap="RdYlGn", vmin=0, vmax=1, annot=True, fmt=".2f",
    linewidths=0.5, linecolor="white", annot_kws={"size": 7},
    ax=ax
)
ax.set_title("Cosine Similarity Heatmap — 20 Sample Articles",
             fontsize=14, fontweight="bold", pad=12)
plt.xticks(rotation=45, ha="right", fontsize=8)
plt.yticks(rotation=0, fontsize=8)
plt.tight_layout()
plt.savefig("similarity_heatmap.png", dpi=150, bbox_inches="tight")
plt.show()
print("Cosine similarity heatmap saved.")

In [ ]:
# Step 11: Word vector similarity — top-10 similar words for key terms
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

probe_words = ["election", "cricket", "software"]

for ax, word in zip(axes, probe_words):
    if word not in w2v_model.wv:
        ax.set_title(f"'{word}' not in vocab")
        continue
    similar = w2v_model.wv.most_similar(word, topn=10)
    words, scores = zip(*similar)
    colors = plt.cm.Blues(np.linspace(0.4, 0.9, len(words)))
    ax.barh(list(words)[::-1], list(scores)[::-1], color=colors[::-1])
    ax.set_xlabel("Cosine Similarity")
    ax.set_title(f"Words similar to '{word}'", fontweight="bold")
    ax.set_xlim(0, 1)
    ax.grid(axis="x", linestyle="--", alpha=0.4)

plt.suptitle("Top-10 Similar Words (Word2Vec)", fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
plt.savefig("word_similarity_bars.png", dpi=150, bbox_inches="tight")
plt.show()

In [ ]:
# Step 12: Evaluation — Intra vs Inter category similarity
categories = df["category"].unique()

intra_sims = {}  # avg similarity WITHIN a category
inter_sims = {}  # avg similarity ACROSS different categories

for cat in categories:
    cat_idx   = df[df["category"] == cat].index.tolist()
    other_idx = df[df["category"] != cat].index.tolist()

    cat_vecs   = sentence_vectors[cat_idx]
    other_vecs = sentence_vectors[other_idx]

    if len(cat_idx) > 1:
        sim_intra = cosine_similarity(cat_vecs)
        np.fill_diagonal(sim_intra, 0)
        intra_sims[cat] = sim_intra.sum() / (len(cat_idx) * (len(cat_idx) - 1))
    
    inter_sims[cat] = cosine_similarity(cat_vecs, other_vecs).mean()

print("Category Embedding Quality (higher intra & lower inter = better clusters)")
print(f"{'Category':<15} {'Intra-sim':>12} {'Inter-sim':>12} {'Separation':>12}")
print("─" * 55)
for cat in categories:
    intra = intra_sims.get(cat, 0)
    inter = inter_sims.get(cat, 0)
    sep   = intra - inter
    print(f"{cat:<15} {intra:>12.4f} {inter:>12.4f} {sep:>12.4f}")

In [ ]:
# Step 13: Pipeline Summary
print("=" * 60)
print("     SEMANTIC SEARCH & RECOMMENDATION SYSTEM — SUMMARY")
print("=" * 60)
print(f"  Dataset         : BBC News (English)")
print(f"  Articles        : {len(df)} ({df['category'].nunique()} categories)")
print(f"  Preprocessing   : Lowercase + Punct removal + Stopword filter")
print(f"  Embedding Model : Word2Vec Skip-Gram (trained from scratch)")
print(f"  Vocabulary      : {len(w2v_model.wv.key_to_index)} unique tokens")
print(f"  Vector Size     : {w2v_model.vector_size} dimensions")
print(f"  Sentence Vecs   : Mean pooling of word vectors")
print(f"  Search Method   : Cosine Similarity ranking")
print(f"  Visualization   : PCA + t-SNE + Cosine Heatmap + Word Bars")
print("=" * 60)
print("  ✅ Semantic Search  — working")
print("  ✅ Recommendation   — working")
print("  ✅ PCA visualization — saved")
print("  ✅ t-SNE visualization — saved")
print("  ✅ Similarity heatmap  — saved")
print("=" * 60)